#  Feature Engineering Demo (Travel Purchase)

This tutorial walks through an end-to-end workflow using the **latest Feature Engineering Declarative API (BETA)** [AWS](https://docs.databricks.com/aws/en/machine-learning/feature-store/declarative-apis) on Databricks.

We’ll revisit the same use case — a **Travel Agency Recommender Model** — designed to increase revenue by personalizing travel and hotel offers based on each user’s likelihood to make a purchase.

---

 

By the end of this notebook, you’ll have a complete, production-ready feature engineering workflow — from data exploration to model scoring — all defined **declaratively and reproducibly**.


## End-to-End Workflow

1. **Explore & preprocess** raw tables registered in Unity Catalog  
2. **Define feature sources** using `DeltaTableSource`  
3. **Declare time-window features** with `create_feature`  
4. **Assemble a unified training set** using point-in-time features 
5. **Train an ML model** to predict the `purchased` label  
6. **Reuse the same features** for batch inference  

---

## Requirements

- **Databricks Runtime:** 17.0 LTS ML or above   
- **Serverless:** Not yet supported (coming soon)  

---

## Expected Output

By the end of this notebook, you will:
- View registered **feature tables** and **feature functions** in Unity Catalog  
- Generate a **training dataset** enriched time-window features  
- Train and evaluate a simple model predicting travel purchase likelihood  
- Perform **batch inference** using the same declarative features  



In [0]:
# Need to install the feature engineering library
%pip install databricks-feature-engineering>=0.14.0
dbutils.library.restartPython()

In [0]:
from databricks.feature_engineering import FeatureEngineeringClient
from databricks.feature_engineering.entities import (
    DeltaTableSource, Sum, Avg, SlidingWindow, Count, ContinuousWindow, TumblingWindow, StddevSamp, Max, Min, Window,
)
from datetime import timedelta
fe = FeatureEngineeringClient()

## 1. Explore raw data
We start from two tables in Unity Catalog:
- `travel_purchase`: event-level user interactions and purchases
- `user_demography`: static attributes keyed by `user_id`


In [0]:
%sql SELECT * FROM travel_purchase limit 10

### 1.1 Preprocess Raw Source Table

Before we define features, every ML workflow begins with **data preprocessing** — the foundation of a reliable model.  
In the declarative feature engineering API, preprocessing ensures that the raw source data is clean, typed correctly, and ready for numerical aggregations.

In a typical ML lifecycle, preprocessing includes:
- **Data cleaning** — handling missing values, outliers, or inconsistent formats  
- **Feature type casting** — converting categorical or boolean fields into numeric forms suitable for aggregations  
- **Key and timestamp alignment** — ensuring each record can be uniquely identified and temporally ordered  

For this travel purchase use case, our raw behavioral event table (e.g., `user_clicks`) will be lightly transformed to meet the declarative API’s current expectations for **simple, numeric inputs**:

- Cast `clicked` (**BOOLEAN**) → `clicked_int` (**INT**) so it can be used in numeric aggregations  
- Add a new column `impression` (constant value = 1) to enable **impression counts** using `sum()` aggregations  
- Keep the original **primary keys** (`user_id`, `session_id`) and **timestamp column** (`event_time`) unchanged  

These preprocessing steps create a consistent, analytics-friendly schema — enabling the `DeltaTableSource` and `create_feature()` API to compute robust and interpretable time-window features downstream.



In [0]:
from pyspark.sql import functions as F, Window as W
cleaned_df = (
    spark.table(f"{catalog}.{schema}.travel_purchase")
         .withColumn("clicked_int", F.col("clicked").cast("int"))
         .withColumn("impression", F.lit(1))  # for Count()
)

cleaned_df.write.format("delta").mode("overwrite").saveAsTable(
    f"{catalog}.{schema}.travel_purchase_clean"
)

## 2. Define Feature Sources with `DeltaTableSource`

In the declarative workflow, a **feature source** defines *where* the data comes from and *how* it should be interpreted for feature computation.  
Each `DeltaTableSource` acts as a declarative reference to a Unity Catalog table — specifying its catalog, schema, keys, and time column — so features can later be derived reproducibly and governed under Unity Catalog lineage.

For this use case, we define **two feature sources**:

1. **User behavior features** — keyed by `user_id`, representing how each user interacts with travel listings  
2. **Destination features** — keyed by `destination_id`, capturing aggregated statistics about travel destinations  

Both use `event_time` as the **time column** to support time-windowed feature computations such as rolling averages or click counts.

<img src="https://github.com/databricks-demos/dbdemos-resources/blob/main/images/product/feature_store/2025Q4_04_image7.png?raw=true" style="float: right" width="650px" />
 
### 2.1 Example

The example below demonstrates how to define a feature source from a Unity Catalog table `main.analytics.user_events`:

```python
from databricks.feature_engineering.entities import DeltaTableSource

user_behavior_source = DeltaTableSource(
    catalog_name="main",        # Catalog name
    schema_name="analytics",    # Schema name
    table_name="user_events",   # UC table name
    entity_columns=["user_id"], # Primary entity key(s)
    timeseries_column="event_time"  # Timestamp column for time-window features
)


In [0]:
# Source with entity keys at user_id level
behavior_source = DeltaTableSource(
    catalog_name=catalog,
    schema_name=schema,
    table_name="travel_purchase_clean",
    entity_columns=["user_id"],
    timeseries_column="ts",
)

# Source with entity keys at destination_id level
destination_source = DeltaTableSource(
    catalog_name=catalog,
    schema_name=schema,
    table_name="travel_purchase_clean",
    entity_columns=["destination_id"],
    timeseries_column="ts",
)


## 3. Declare Time-Window Features with `create_feature`

After defining feature sources, the next step is to **declare time-window features** — metrics that summarize user or system behavior over a defined period (e.g., last 7 days, last 30 days).  
With the new declarative API, you can build these aggregations directly using `FeatureEngineeringClient.create_feature()`, which automatically handles validation, metadata registration, and lineage tracking in Unity Catalog.








### 3.1 Concept

A **time-window feature** aggregates a column of values across a rolling or continuous time window, producing features such as:
- “Total number of clicks in the past 7 days”  
- “Average booking price over the last 30 days”  
- “Maximum spend in the last session”

These temporal summaries capture recent trends in user behavior and are often the most predictive signals for recommender systems, fraud detection, or personalization models.


###  3.2 Supported Aggregation Functions

> **Note:** All functions operate over the specified aggregation window.  
> The window type (e.g., *sliding* or *continuous*) is defined in the feature pipeline configuration.

| Function | Shorthand | Description | Example Use Case |
|-----------|------------|-------------|------------------|
| `Sum()` | `"sum"` | Total of all values within the window | Total minutes streamed per user |
| `Avg()` / `Mean()` | `"avg"`, `"mean"` | Average value over the window | Mean transaction amount |
| `Count()` | `"count"` | Number of records | Number of logins per user |
| `Min()` | `"min"` | Minimum value observed | Lowest heart rate recorded |
| `Max()` | `"max"` | Maximum value observed | Largest basket size per session |

For additional supported functions (e.g., `stddev`, `count_distinct`, `approx_percentile`), see the  
[Feature Engineering API documentation](https://docs.databricks.com/aws/en/machine-learning/feature-store/declarative-apis#supported-functions).

### 3.3 Example

The recommended and most robust approach is to use the  
**`FeatureEngineeringClient.create_feature()`** method, which performs comprehensive validation and ensures correct feature registration.

```python
from databricks.feature_engineering import FeatureEngineeringClient
from databricks.feature_engineering.entities import DeltaTableSource, TimeWindow

fe = FeatureEngineeringClient()

feature = fe.create_feature(
    source=DeltaTableSource(...),          # Required: DeltaTableSource or DataFrameSource
    inputs=["clicked_int"],                # Required: List of input columns
    function="sum",                        # Required: Aggregation function
    time_window=TimeWindow("7d"),          # Required: Aggregation window (e.g., 7 days)
    catalog_name="main",                   # Required: Target catalog
    schema_name="features",                # Required: Target schema
    name="user_clicks_7d",                 # Optional: Feature name (auto-generated if omitted)
    description="Total user clicks in past 7 days",  # Optional: Feature description
    filter_condition=None                  # Optional: SQL filter for source data
)


### 3.4 Managing Feature Functions in Unity Catalog

When you create a time-window feature using `create_feature()`, Databricks automatically registers a **feature function** in **Unity Catalog** under the specified schema.  
This function encapsulates the feature’s logic (e.g., input column, aggregation, and window definition) and allows consistent reuse across training and inference pipelines.

If you **omit the `name` parameter** during creation, the system will automatically generate a descriptive name based on:
- The **input column name**
- The **aggregation function**
- The **time-window specification**

<img src="https://github.com/databricks-demos/dbdemos-resources/blob/main/images/product/feature_store/2025Q4_04_image3.png?raw=true" style="float: right" width="350px" />

Over time, you may want to remove unused or experimental feature functions to keep your schema organized.  
To delete a previously created feature function, use:

```python
fe.delete_feature_spec("<feature_function_name>")
```

Pleae regularly clean up old or deprecated feature functions to keep your schema tidy, maintainable, and production-ready.


### 3.5 Time Windows in Declarative Feature Engineering

Time windows define **how far back in time** data is aggregated when computing a feature.  
The Declarative Feature Engineering API supports multiple window types, each designed for different analytical or operational needs.
<img src="https://github.com/databricks-demos/dbdemos-resources/blob/main/images/product/feature_store/2025Q4_04_image5.png?raw=true" style="float: right" width="550px" />


- **Tumbling Window**: Tumbling windows divide time into **non-overlapping, consecutive intervals**, where each record belongs to exactly one window.  
They are best suited for **periodic summaries** — for example, total bookings per week or average spend per month.

- **Sliding Window**:Sliding windows define **overlapping intervals** that advance by a configurable step (the slide interval).  
They are used for **rolling or moving aggregations**, such as a 7-day rolling average or a 30-day engagement rate recalculated daily.

- **Continuous Window**: Continuous windows compute feature values **at a specific evaluation time**, aggregating data over a defined duration ending at that point.  
They are ideal for **real-time** or **point-in-time** computations, such as streaming or online inference.  


#### Define User behavior features (per `user_id`)
- `mean_price_7d`: 7-day rolling average of `price`
- `mean_price_30d`: 30-day rolling average of `price`
- `user_7d_clicks` : 7-day rolling sum of `clicked_int`
- `user_7d_impression`: 7-day rolling sum of `impression`
- `price_volatility_30d`: 30-day StddevSamp of `price`

In [0]:
user_behavior_features = [

    # 1) Short-horizon price intent (SLIDING)
    fe.create_feature(
        catalog_name=catalog, schema_name=schema,
        #name="mean_price_7d",
        source=behavior_source,
        inputs=["price"],
        function=Avg(),
        time_window=SlidingWindow(
            window_duration=timedelta(days=7),
            slide_duration=timedelta(days=1),
        ),
    ),

    # 2) Stable price intent (CONTINUOUS)
    fe.create_feature(
        catalog_name=catalog, schema_name=schema,
        #name="mean_price_30d_cont",
        source=behavior_source,
        inputs=["price"],
        function=Avg(),
        time_window=ContinuousWindow(
            window_duration=timedelta(days=30),
        ),
    ),

    # 3) User engagement via clicks (TUMBLING)
    fe.create_feature(
        catalog_name=catalog, schema_name=schema,
        #name="user_clicks_7d_tumbling",
        source=behavior_source,
        inputs=["clicked_int"],           # 0/1
        function=Sum(),
        time_window=TumblingWindow(
            window_duration=timedelta(days=7),
        ),
    ),

    # 4) User activity via impressions (SLIDING)
    fe.create_feature(
        catalog_name=catalog, schema_name=schema,
        #name="user_activity_7d",
        source=behavior_source,
        inputs=["impression"],            # always 1
        function=Sum(),
        time_window=SlidingWindow(
            window_duration=timedelta(days=7),
            slide_duration=timedelta(days=1),
        ),
    ),


    # 5) User variability signal (SLIDING)
    fe.create_feature(
        catalog_name=catalog, schema_name=schema,
        #name="price_volatility_30d",
        source=behavior_source,
        inputs=["price"],
        function=StddevSamp(),
        time_window=SlidingWindow(
            window_duration=timedelta(days=30),
            slide_duration=timedelta(days=1),
        ),
    ),
]

#### For Debugging and Exploration - `compute_feature()` 

The `compute_feature()` method allows you to **preview feature values** by applying the declared feature logic directly to the source data.  
It’s primarily designed for **debugging, validation, and exploratory analysis** — helping you verify that your feature definitions and aggregations behave as expected before materializing them with a pipeline.

Typical uses include:
- Inspecting computed feature values joined back to the raw source table  
- Validating aggregation correctness or window logic  
- Quickly testing transformations in development or notebook sessions  

**Note**: `compute_feature()` is not intended for production use.
It performs on-demand computation and does not persist or manage feature lineage in Unity Catalog.
For production workflows, always use `create_pipeline()` to materialize features to Delta tables with governance, versioning, and scheduling.


In [0]:
feature_values = fe.compute_features(features=user_behavior_features)
feature_values.display()

#### Destination features (per `destination_id`)
- `sum_clicks_7d`: 7-day rolling sum of `clicked_int` (number of clicks)
- `sum_impressions_7d`: 7-day rolling sum of `impression` (number of impressions)
- `dest_mean_price_7d`: 7-day rolling average of `price`
- `dest_mean_price_30d_cont`: 30-day rolling average of `price`


In [0]:
destination_features = [

    # 6) Destination popularity via clicks (SLIDING)
    fe.create_feature(
        catalog_name=catalog, schema_name=schema,
        name="dest_sum_clicks_7d",
        source=destination_source,
        inputs=["clicked_int"],            # 0/1
        function=Sum(),
        time_window=SlidingWindow(
            window_duration=timedelta(days=7),
            slide_duration=timedelta(days=1),
        ),
    ),

    # 7) Destination exposure via impressions (TUMBLING)
    fe.create_feature(
        catalog_name=catalog, schema_name=schema,
        name="dest_sum_impressions_7d_tumbling",
        source=destination_source,
        inputs=["impression"],             # 1
        function=Sum(),
        time_window=TumblingWindow(
            window_duration=timedelta(days=7),
        ),
    ),

    # 8) Recent destination pricing (SLIDING)
    fe.create_feature(
        catalog_name=catalog, schema_name=schema,
        name="dest_mean_price_7d",
        source=destination_source,
        inputs=["price"],
        function=Avg(),
        time_window=SlidingWindow(
            window_duration=timedelta(days=7),
            slide_duration=timedelta(days=1),
        ),
    ),

    # 9) Seasonal destination pricing (CONTINUOUS)
    fe.create_feature(
        catalog_name=catalog, schema_name=schema,
        name="dest_mean_price_30d_cont",
        source=destination_source,
        inputs=["price"],
        function=Avg(),
        time_window=ContinuousWindow(
            window_duration=timedelta(days=30),
        ),
    ),
]

In [0]:
feature_values = fe.compute_features(features=destination_features)
feature_values.display()

## 4. Model Training with Declarative Feature Engineering (Point-in-Time Correct Features)
<img src="https://github.com/databricks-demos/dbdemos-resources/blob/main/images/product/feature_store/2025Q4_04_image10.png?raw=true" style="float: right" width="550px" />
In this section, we demonstrate how to build a machine learning model using the Declarative Feature Engineering API on Databricks.  

### 4.1 `fe.create_training_set()`  
This API automatically performs **point-in-time feature computation**, ensuring that every feature is joined using the correct values **as of the event timestamp** (for example, at purchase time or click time).  

This eliminates common ML pitfalls such as data leakage, incorrect lookback windows, or manually managing temporal joins.


### 4.2 About Non-Aggregation/Static Features (Current Limitation)

The current release of the Declarative Feature Engineering API focuses on **time-window aggregation features** (e.g., sliding windows, tumbling windows, continuous windows).  
At this time, the API **does not yet support transformation-only, non-aggregate features** such as:

- gender 
- age
- billing state, etc

We intentionally keep the feature set clean and numeric, based purely on **rolling aggregations**, which are both powerful and easy to interpret. As Databricks adds support for **transformation features** in future releases,**we will seamlessly update this notebook** to include richer engineered features.

In the cells that follow, we:

1. Use `fe.create_training_set()` to assemble a training dataset with **point-in-time accurate** rolling features.
2. Convert the training set into Pandas for classical ML workflows.
3. Train a simple, interpretable ML model (RandomForest).
4. Log the model and its feature lineage using `fe.log_model`, ensuring reproducibility and traceability.

By the end, you will have a fully traceable ML model whose predictions can always be reproduced from the underlying **declarative feature definitions** — a cornerstone of production-grade ML systems.

In [0]:
id_and_label = spark.table('travel_purchase').select("id", "purchased", "destination_id", "user_id", "ts").limit(1000)
display(id_and_label.limit(10))

In [0]:
print(f" Label distribution in training data:")
id_and_label.groupBy("purchased").count().show()

In [0]:
# 1) Create training set with point-in-time feature lookup
training_set = fe.create_training_set(
        name="travel_purchase_training_set_rolling_only",
        catalog_name=catalog,
        schema_name=schema,
        df=id_and_label,            
        features=user_behavior_features + destination_features,    # rolling features (user + destination)
        label="purchased",
        exclude_columns=["user_id", "destination_id", "ts"],  
    )

print("Loading training DataFrame...")
training_df = training_set.load_df()
print(f"Spark DF rows: {training_df.count()}, cols: {len(training_df.columns)}")
display(training_df.limit(10))

In [0]:
 # 2) Convert to Pandas and prepare features/label
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

training_pd = training_df.toPandas()

# Drop label + keys/time if present
drop_cols = [c for c in ["purchased", "user_id", "destination_id", "ts"] if c in training_pd.columns]
X = training_pd.drop(columns=drop_cols, errors="ignore").copy()
y = training_pd["purchased"].astype(int).values.ravel()

# Simple numeric fill for safety (rolling features are numeric)
X = X.fillna(0)

x_train, x_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

print(f"x_train: {x_train.shape}, y_train: {y_train.shape}")
print(f"x_val:   {x_val.shape},   y_val:   {y_val.shape}")


In [0]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, roc_auc_score

In [0]:
# Define, train, evaule and log the model
import mlflow
mlflow.set_experiment("/dbdemos_travel_rec")

model_name = "dbdemos_fs_travel_dapi_rolling_only"
model_full_name = f"{catalog}.{schema}.{model_name}"

with mlflow.start_run(run_name="travel_rec_declarative_features") as run:
    
    print("Training RandomForestClassifier inside MLflow run...")

    model = RandomForestClassifier(
        n_estimators=200,
        max_depth=None,
        min_samples_leaf=2,
        random_state=42,
        n_jobs=-1
    )
    model.fit(x_train, y_train)

    # Evaluate
    y_pred = model.predict(x_val)
    if len(model.classes_) > 1:
        y_prob = model.predict_proba(x_val)[:, 1]
    else:
        y_prob = np.zeros_like(y_pred, dtype=float)

    auc = roc_auc_score(y_val, y_prob) if len(np.unique(y_val)) > 1 else 0.5

    print(" Validation Performance:")
    print(classification_report(y_val, y_pred, digits=3))
    print(f" AUC (val): {auc:.3f}")

    # Log metrics + artifacts
    mlflow.log_metric("val_auc", float(auc))
    mlflow.log_metric("num_features", int(X.shape[1]))
    mlflow.log_metric("num_rows", int(len(training_pd)))

    # Feature importance
    importances = model.feature_importances_
    fi = dict(sorted(zip(list(X.columns), importances), key=lambda x: x[1], reverse=True))
    mlflow.log_dict(fi, "feature_importance.json")

    print("\n Top Feature Importances:")
    for k, v in list(fi.items())[:15]:
        print(f" • {k}: {v:.4f}")


    # Log the model *with declarative feature lineage*
    print("\n Logging model to Unity Catalog with feature lineage...")

    fe.log_model(
        model=model,
        artifact_path="model",
        flavor=mlflow.sklearn,
        training_set=training_set,        # links DFE features → model lineage
        registered_model_name=model_full_name
    )

print(f"Registered model: {model_full_name}")



### 4.3 Model lineage

Lineage is automatically captured and visible within Unity Catalog. It tracks all tables up to the model created.

<img src="https://github.com/databricks-demos/dbdemos-resources/blob/main/images/product/feature_store/2025Q4_04_image9.png?raw=true" style="float: right" width="650px" />

This makes it easy to track all your data usage, and downstream impact. If some PII information got leaked, or some incorrect data is loaded and detected by the Lakehouse Monitoring, it's then easy to track the potential impact.

Note that this not only includes table and model, but also Notebooks, Dashboard, Jobs triggering the run etc.

In [0]:
from mlflow import MlflowClient

mlflow_client = MlflowClient()
# Use the MlflowClient to get a list of all versions for the registered model in Unity Catalog
all_versions = mlflow_client.search_model_versions(f"name='{model_full_name}'")
# Sort the list of versions by version number and get the latest version
latest_version = max([int(v.version) for v in all_versions])
# Use the MlflowClient to get the latest version of the registered model in Unity Catalog
latest_model = mlflow_client.get_model_version(model_full_name, str(latest_version))

In [0]:
#Move it in Production
production_alias = "production"
if len(latest_model.aliases) == 0 or latest_model.aliases[0] != production_alias:
  print(f"updating model {latest_model.version} to Production")
  mlflow_client.set_registered_model_alias(model_full_name, production_alias, version=latest_version)

### 4.4 Running batch inferences

We are now ready to run inferences.

In a real world setup, we would receive new data from our customers and have our job incrementally refreshing our customer features running in parallel. 

To make the predictions, all we need to have is the primary keys for each feature loopup table. Feature Engineering in UC will automatically do the lookup for us as defined in the training steps.

This is one of the great outcome using the Feature Engineering in UC: you know that your features will be used the same way for inference as training because it's being saved with your feature store metadata.

In [0]:
# Load the ids we want to forecast
## For sake of simplicity, we will just predict using the same ids as during training, but this could be a different pipeline
id_to_forecast = spark.table('travel_purchase').select("id","user_id","destination_id", "ts")

# Use result_type="double" to get probability scores (values between 0 and 1)
# Or use result_type="int" to get class predictions (0 or 1)
scored_df = fe.score_batch(model_uri=f"models:/{model_full_name}@{production_alias}", df=id_to_forecast, result_type="double")
display(scored_df)

## 5 Materialize Features with `create_pipeline()`

After declaring individual features, use a **feature pipeline** to materialize them into an **offline feature table** for training and batch inference.  
`FeatureEngineeringClient.create_pipeline()` builds a reproducible job that **backfills**, **schedules incremental runs**, and **writes managed Delta tables** in Unity Catalog.

<img src="https://github.com/databricks-demos/dbdemos-resources/blob/main/images/product/feature_store/2025Q4_04_image2.png?raw=true" style="float: right" width="650px" />



### 5.1 What it does
- **Groups compatible features** into efficient execution plans
- **Backfills** from a defined `start_time` (or auto-infers from source data)
- **Schedules** recurring refreshes (e.g., hourly/daily) for rolling windows
- **Materializes** results to a governed UC table (catalog/schema/prefix)

### 5.2 BETA Limitation

Today, the Feature Pipeline (in public preview) only supports **SlidingWindow** features for **incremental scheduling and refresh**.

This means:

- `ContinuousWindow` and `TumblingWindow` **will work in create_training_set()**,  
  but **cannot yet be incrementally materialized** using a scheduled pipeline.
- A Feature Pipeline will only accept features with **SlidingWindow** time windows.

This limitation is temporary and will be expanded in future releases.


### 5.2 Example

```python
FeatureEngineeringClient.create_pipeline(
    features,                # List[Feature] – features to materialize
    offline_store_config,    # OfflineStoreConfig – UC catalog/schema/table prefix
    schedule=None,           # Optional[CronSchedule] – refresh cadence
    start_time=None          # Optional[datetime] – earliest time to backfill from
)



In [0]:
# Filter only features with SlidingWindow (required for scheduled pipelines)
Slidingfeatures = [
    f for f in (user_behavior_features + destination_features)
    if isinstance(f.time_window, SlidingWindow)
]

print(f"Found {len(Slidingfeatures)} features with SlidingWindow:")
for f in Slidingfeatures:
    print(f"  • {f.name}")

In [0]:
from databricks.feature_engineering.entities import OfflineStoreConfig
from databricks.feature_engineering import CronSchedule
offline_config = OfflineStoreConfig(
    catalog_name=catalog,
    schema_name=dbName,
    table_name_prefix="travel_rec"
)
fe.create_pipeline(
    features=Slidingfeatures,
    offline_store_config=offline_config,
    schedule = None,
    #schedule=CronSchedule(quartz_cron_expression="0 0 * * * ?", timezone_id="UTC"),   # run daily at 00:00,
    start_time=None,
)
print ("Pipeline created and executed!")

## 🎉 Wrapping Up: What's Next for Your Feature Engineering Workflow?

Congratulations — you’ve now completed an end-to-end walkthrough of Databricks **Declarative Feature Engineering**, including:

- Defining time-aware rolling features using the new DFE API  
- Generating **point-in-time correct** training sets with `fe.create_training_set()`  
- Training an ML model with full **feature lineage**  
- Materializing features using a **Feature Pipeline** for offline batch use cases

This workflow provides a strong foundation for reproducible, governable, and production-ready feature engineering on the Lakehouse.

## Next Steps: Online Feature Serving with LakeBase

If your workspace have a **LakeBase instance** available, you can take your solution to the next level:

### Publish features to an **Online Feature Store**  
Using your existing feature definitions, you can push features to LakeBase for **low-latency, real-time inference** — exactly like we demonstrated in *Notebook 2*.

This gives you:

- Millisecond-level feature lookups  
- Unified online–offline consistency  
- A clean path to powering real-time recommendation systems, fraud models, and more

## Looking Ahead: Unified Offline + Online Publishing in Pipelines

The current Feature Pipeline (in BETA) focuses on **offline feature materialization**. However, an upcoming release will add support for:

### `offline_config` and `online_config`  
within `create_pipeline()` — enabling:

- Offline feature materialization (MV tables)
- Online feature publishing (LakeBase)
- Both **from the same declarative pipeline**


## 